In [1]:
# import pandas as pd
import polars as pl
import os

In [2]:
from datetime import datetime, timedelta

schema_overrides = {
    "start_station_name": pl.Utf8,
    "end_station_name": pl.Utf8,
    "start_lat": pl.Float64,
    "start_lng": pl.Float64,
    "end_lat": pl.Float64,
    "end_lng": pl.Float64,
}

null_values = ["MTL-ECO5-03"]

daily_rental_raw = pl.read_csv(
    "./data/daily_rental_raw.csv",
    schema_overrides=schema_overrides,
    ignore_errors=True,
    null_values=null_values
)

print("Schema:",daily_rental_raw.schema)
print(f"Daily rental Shape: {daily_rental_raw.shape}")


Schema: Schema([('ride_id', String), ('rideable_type', String), ('started_at', String), ('ended_at', String), ('start_station_name', String), ('start_station_id', Float64), ('end_station_name', String), ('end_station_id', Float64), ('start_lat', Float64), ('start_lng', Float64), ('end_lat', Float64), ('end_lng', Float64), ('member_casual', String)])
Daily rental Shape: (18007255, 13)


In [3]:
print(daily_rental_raw.select(["started_at", "ended_at","start_station_id","end_station_id"]).head())

shape: (5, 4)
┌─────────────────────┬─────────────────────┬──────────────────┬────────────────┐
│ started_at          ┆ ended_at            ┆ start_station_id ┆ end_station_id │
│ ---                 ┆ ---                 ┆ ---              ┆ ---            │
│ str                 ┆ str                 ┆ f64              ┆ f64            │
╞═════════════════════╪═════════════════════╪══════════════════╪════════════════╡
│ 2020-12-02 09:10:36 ┆ 2020-12-02 09:24:12 ┆ 31114.0          ┆ 31242.0        │
│ 2020-12-31 12:46:29 ┆ 2020-12-31 14:01:07 ┆ 31606.0          ┆ 31290.0        │
│ 2020-12-31 12:47:03 ┆ 2020-12-31 14:01:04 ┆ 31606.0          ┆ 31290.0        │
│ 2020-12-29 13:50:51 ┆ 2020-12-29 14:12:22 ┆ 31054.0          ┆ 31010.0        │
│ 2020-12-27 12:30:28 ┆ 2020-12-27 12:36:12 ┆ 31920.0          ┆ 31024.0        │
└─────────────────────┴─────────────────────┴──────────────────┴────────────────┘


In [3]:
def cast_data(daily_rental_raw):
    """
    Processes the raw daily rental data by:
    - Converting 'started_at' and 'ended_at' columns to datetime in a America/New_York timezone.
    - Casting 'start_station_id' and 'end_station_id' to Int64.
    
    Args:
        daily_rental_raw (pl.DataFrame): The raw daily rental data.
    
    Returns:
        pl.DataFrame: The processed daily rental data.
    """
    return daily_rental_raw.with_columns([
        # Convert 'started_at' to datetime and adjust time zone
        pl.col("started_at")
          .str.to_datetime("%Y-%m-%d %H:%M:%S", strict=False)
          .dt.convert_time_zone("America/New_York")
          .alias("started_at"),
        
        # Convert 'ended_at' to datetime and adjust time zone
        pl.col("ended_at")
          .str.to_datetime("%Y-%m-%d %H:%M:%S", strict=False)
          .dt.convert_time_zone("America/New_York")
          .alias("ended_at"),
        
        # Cast 'start_station_id' to Int64
        pl.col("start_station_id").cast(pl.Int64, strict=False).alias("start_station_id"),
        
        # Cast 'end_station_id' to Int64
        pl.col("end_station_id").cast(pl.Int64, strict=False).alias("end_station_id"),
    ])


In [4]:
# Assuming `daily_rental_raw` is your raw data
daily_rental_casted = cast_data(daily_rental_raw)

# Check the result
print(daily_rental_casted.select(["started_at", "ended_at","start_station_id","end_station_id"]).head())

shape: (5, 4)
┌─────────────────────────┬────────────────────────────────┬──────────────────┬────────────────┐
│ started_at              ┆ ended_at                       ┆ start_station_id ┆ end_station_id │
│ ---                     ┆ ---                            ┆ ---              ┆ ---            │
│ datetime[μs,            ┆ datetime[μs, America/New_York] ┆ i64              ┆ i64            │
│ America/New_York]       ┆                                ┆                  ┆                │
╞═════════════════════════╪════════════════════════════════╪══════════════════╪════════════════╡
│ 2020-12-02 04:10:36 EST ┆ 2020-12-02 04:24:12 EST        ┆ 31114            ┆ 31242          │
│ 2020-12-31 07:46:29 EST ┆ 2020-12-31 09:01:07 EST        ┆ 31606            ┆ 31290          │
│ 2020-12-31 07:47:03 EST ┆ 2020-12-31 09:01:04 EST        ┆ 31606            ┆ 31290          │
│ 2020-12-29 08:50:51 EST ┆ 2020-12-29 09:12:22 EST        ┆ 31054            ┆ 31010          │
│ 2020-12-27 07:

In [7]:
# daily_rental_df = pl.DataFrame(daily_rental)
print("daily_rental schema:",daily_rental_casted.schema)
print(f"Daily rental Shape: {daily_rental_casted.shape}")

daily_rental schema: Schema([('ride_id', String), ('rideable_type', String), ('started_at', Datetime(time_unit='us', time_zone='America/New_York')), ('ended_at', Datetime(time_unit='us', time_zone='America/New_York')), ('start_station_name', String), ('start_station_id', Int64), ('end_station_name', String), ('end_station_id', Int64), ('start_lat', Float64), ('start_lng', Float64), ('end_lat', Float64), ('end_lng', Float64), ('member_casual', String)])
Daily rental Shape: (18007255, 13)


In [7]:
def inspect_partial_missing_station_info(df: pl.DataFrame, prefix: str) -> pl.DataFrame:
    # Filter rows where either station_id or station_name is missing
    partial_missing = df.filter(
        (pl.col(f"{prefix}station_id").is_null() & pl.col(f"{prefix}station_name").is_not_null()) |
        (pl.col(f"{prefix}station_id").is_not_null() & pl.col(f"{prefix}station_name").is_null())
    )
    # Select relevant columns for inspection
    return partial_missing.select(
        pl.col(f"{prefix}station_id"),
        pl.col(f"{prefix}station_name"),
        pl.col(f"{prefix}lat"),
        pl.col(f"{prefix}lng")
    )

# Inspect for both start_ and end_ stations
partial_missing_start_station_info = inspect_partial_missing_station_info(daily_rental_casted, "start_")
partial_missing_end_station_info = inspect_partial_missing_station_info(daily_rental_casted, "end_")

# Print the results for partial missing station info
print("Partial Missing Start Station Info:")
print(partial_missing_start_station_info)

print("\nPartial Missing End Station Info:")
print(partial_missing_end_station_info)


Partial Missing Start Station Info:
shape: (0, 4)
┌──────────────────┬────────────────────┬───────────┬───────────┐
│ start_station_id ┆ start_station_name ┆ start_lat ┆ start_lng │
│ ---              ┆ ---                ┆ ---       ┆ ---       │
│ i64              ┆ str                ┆ f64       ┆ f64       │
╞══════════════════╪════════════════════╪═══════════╪═══════════╡
└──────────────────┴────────────────────┴───────────┴───────────┘

Partial Missing End Station Info:
shape: (1_055, 4)
┌────────────────┬─────────────────────────────────┬─────────┬─────────┐
│ end_station_id ┆ end_station_name                ┆ end_lat ┆ end_lng │
│ ---            ┆ ---                             ┆ ---     ┆ ---     │
│ i64            ┆ str                             ┆ f64     ┆ f64     │
╞════════════════╪═════════════════════════════════╪═════════╪═════════╡
│ null           ┆ 17th & P St NW                  ┆ 38.91   ┆ -77.04  │
│ null           ┆ Connecticut Ave & R St NW       ┆ 38.91   ┆ 

In [8]:
def count_partial_missing_station_info(df: pl.DataFrame, prefix: str) -> dict:
    # Count rows where station_id is missing but station_name is not null
    missing_id_count = df.filter(
        pl.col(f"{prefix}station_id").is_null() & pl.col(f"{prefix}station_name").is_not_null()
    ).height

    # Count rows where station_name is missing but station_id is not null
    missing_name_count = df.filter(
        pl.col(f"{prefix}station_id").is_not_null() & pl.col(f"{prefix}station_name").is_null()
    ).height

    return {"missing_id": missing_id_count, "missing_name": missing_name_count}

# Get counts for end_ prefix
end_station_missing_info = count_partial_missing_station_info(daily_rental_casted, "end_")

# Print the results
print("End Station Missing Information:")
print(f"Missing end_station_id (but end_station_name present): {end_station_missing_info['missing_id']}")
print(f"Missing end_station_name (but end_station_id present): {end_station_missing_info['missing_name']}")


End Station Missing Information:
Missing end_station_id (but end_station_name present): 1055
Missing end_station_name (but end_station_id present): 0


In [33]:
station_information= pl.read_csv("data/station_infor.csv").filter(
    (pl.col("lat").is_not_null()) & (pl.col("lon").is_not_null())
)
def fill_missing_station_info_with_null(daily_rental: pl.DataFrame, station_info: pl.DataFrame) -> pl.DataFrame:
    """
    Fill missing station IDs in daily_rental based on station_info, 
    and set station_id to null if station_name is missing in station_info.

    Args:
        daily_rental (pl.DataFrame): The daily rental dataset.
        station_info (pl.DataFrame): The station information dataset.

    Returns:
        pl.DataFrame: Updated daily_rental DataFrame with missing station IDs filled, and unmatched rows set to null.
    """
    # Ensure column names match between daily_rental and station_info
    station_info = station_info.rename({"name": "end_station_name", "short_name": "station_short_name"})

    # Join daily_rental with station_info on station name
    filled_df = daily_rental.join(
        station_info,
        how="left",
        left_on=["end_station_name"],
        right_on=["end_station_name"]
    )

    # Update end_station_id where matches are found, and set unmatched rows to null
    filled_df = filled_df.with_columns(
        pl.when(pl.col("station_short_name").is_not_null())
        .then(pl.col("station_short_name"))
        .otherwise(None)  # Set to null if no match is found
        .alias("end_station_id")
    ).drop(["station_short_name"])  # Drop unnecessary column after filling

    return filled_df



In [11]:
# Find unmatched station names in the daily_rental that don't exist in station_info
unmatched_names = daily_rental_casted.filter(
    ~pl.col("start_station_name").is_in(station_information["name"])
)
print(unmatched_names.select("end_station_name","end_station_id"))


shape: (850_361, 2)
┌────────────────────────────────┬────────────────┐
│ end_station_name               ┆ end_station_id │
│ ---                            ┆ ---            │
│ str                            ┆ i64            │
╞════════════════════════════════╪════════════════╡
│ 18th St & Pennsylvania Ave NW  ┆ 31242          │
│ Georgetown Harbor / 30th St NW ┆ 31215          │
│ 18th St & Pennsylvania Ave NW  ┆ 31242          │
│ Georgetown Harbor / 30th St NW ┆ 31215          │
│ Georgetown Harbor / 30th St NW ┆ 31215          │
│ …                              ┆ …              │
│ null                           ┆ null           │
│ null                           ┆ null           │
│ null                           ┆ null           │
│ null                           ┆ null           │
│ null                           ┆ null           │
└────────────────────────────────┴────────────────┘


In [13]:
cleaned_unmatched_names = unmatched_names.filter(
    ~pl.col("end_station_name").is_in(station_information["name"])
)

# Display the cleaned data
print(cleaned_unmatched_names.select("end_station_name", "end_station_id"))

shape: (116_827, 2)
┌────────────────────────────────┬────────────────┐
│ end_station_name               ┆ end_station_id │
│ ---                            ┆ ---            │
│ str                            ┆ i64            │
╞════════════════════════════════╪════════════════╡
│ Lee Hwy & N Adams St           ┆ 31030          │
│ 8th & K St NE                  ┆ 31660          │
│ 8th & K St NE                  ┆ 31660          │
│ Key Blvd & N Quinn St          ┆ 31027          │
│ Wilson Blvd. & N. Vermont St.  ┆ 31926          │
│ …                              ┆ …              │
│ C & O Canal & Wisconsin Ave NW ┆ 31225          │
│ C & O Canal & Wisconsin Ave NW ┆ 31225          │
│ C & O Canal & Wisconsin Ave NW ┆ 31225          │
│ C & O Canal & Wisconsin Ave NW ┆ 31225          │
│ C & O Canal & Wisconsin Ave NW ┆ 31225          │
└────────────────────────────────┴────────────────┘


In [46]:
def null_condition(prefix):
    return (
        (
            (pl.col(f"{prefix}station_name").is_not_null()) &
            (pl.col(f"{prefix}station_id").is_not_null()) 
        ) & (
            (pl.col(f"{prefix}lat").is_not_null()) &
            (pl.col(f"{prefix}lng").is_not_null())
        ) & (
            (pl.col(f"{prefix}lat").is_not_null()) &
            (pl.col(f"{prefix}lng").is_not_null())
        ) #Filter out rows with disabled/inactive station
    )

filter_condition = (
    (pl.col("started_at").is_not_null()) &
    (pl.col("ended_at").is_not_null()) &
    null_condition("start_") &
    null_condition("end_")
)

# Apply filter
daily_rental_filtered = daily_rental_casted.filter(filter_condition)

In [47]:
def count_nulls(df: pl.DataFrame, columns: list) -> pl.DataFrame:
    null_counts = {
        col: df[col].null_count() for col in columns
    }
    return pl.DataFrame([null_counts])

# Example usage
columns_to_check = [
    "start_station_name", "start_station_id", "start_lat", "start_lng",
    "end_station_name", "end_station_id", "end_lat", "end_lng"
]

# Assuming your DataFrame is named `daily_rental`
nulls_df = count_nulls(daily_rental_filtered, columns_to_check)

print(nulls_df.select("start_station_name", "start_station_id", "start_lat", "start_lng",
    "end_station_name", "end_station_id", "end_lat", "end_lng"))

shape: (1, 8)
┌─────────────┬─────────────┬───────────┬───────────┬─────────────┬────────────┬─────────┬─────────┐
│ start_stati ┆ start_stati ┆ start_lat ┆ start_lng ┆ end_station ┆ end_statio ┆ end_lat ┆ end_lng │
│ on_name     ┆ on_id       ┆ ---       ┆ ---       ┆ _name       ┆ n_id       ┆ ---     ┆ ---     │
│ ---         ┆ ---         ┆ i64       ┆ i64       ┆ ---         ┆ ---        ┆ i64     ┆ i64     │
│ i64         ┆ i64         ┆           ┆           ┆ i64         ┆ i64        ┆         ┆         │
╞═════════════╪═════════════╪═══════════╪═══════════╪═════════════╪════════════╪═════════╪═════════╡
│ 0           ┆ 0           ┆ 0         ┆ 0         ┆ 0           ┆ 0          ┆ 0       ┆ 0       │
└─────────────┴─────────────┴───────────┴───────────┴─────────────┴────────────┴─────────┴─────────┘


In [48]:
# daily_rental_df = pl.DataFrame(daily_rental)
print("daily_rental schema:",daily_rental_filtered.schema)
print(f"Daily rental Shape: {daily_rental_filtered.shape}")

daily_rental schema: Schema([('ride_id', String), ('rideable_type', String), ('started_at', Datetime(time_unit='us', time_zone='America/New_York')), ('ended_at', Datetime(time_unit='us', time_zone='America/New_York')), ('start_station_name', String), ('start_station_id', Int64), ('end_station_name', String), ('end_station_id', Int64), ('start_lat', Float64), ('start_lng', Float64), ('end_lat', Float64), ('end_lng', Float64), ('member_casual', String)])
Daily rental Shape: (12605921, 13)


In [50]:
# Write to CSV
daily_rental_filtered.write_csv("./data/daily_rental_filtered.csv")

In [ ]:
region = pl.read_csv("data/regions.csv")

print("Region schema:",region.schema)
print(f"Region Shape: {region.shape}")


In [7]:
import polars as pl

station_information= pl.read_csv("data/station_infor.csv").filter(
    (pl.col("lat").is_not_null()) & (pl.col("lon").is_not_null())
)

print("station_information schema:",station_information.schema)
print(f"station_information Shape: {station_information.shape}")

station_information.write_csv("./data/station_info.csv")


station_information schema: Schema([('short_name', Int64), ('capacity', Int64), ('region_id', Int64), ('station_id', String), ('lon', Float64), ('name', String), ('lat', Float64)])
station_information Shape: (785, 7)


In [8]:
# Count the number of NULL values in each column
null_counts = station_information.select([pl.col(c).is_null().sum().alias(c + "_null_count") for c in station_information.columns])

# Print the result
print(null_counts)

shape: (51, 7)
┌────────────┬──────────┬───────────┬──────────────────┬────────────┬──────────────────┬───────────┐
│ short_name ┆ capacity ┆ region_id ┆ station_id       ┆ lon        ┆ name             ┆ lat       │
│ ---        ┆ ---      ┆ ---       ┆ ---              ┆ ---        ┆ ---              ┆ ---       │
│ i64        ┆ i64      ┆ i64       ┆ str              ┆ f64        ┆ str              ┆ f64       │
╞════════════╪══════════╪═══════════╪══════════════════╪════════════╪══════════════════╪═══════════╡
│ 31725      ┆ 19       ┆ null      ┆ 1932168474073292 ┆ -76.982464 ┆ 16th St & North  ┆ 38.893062 │
│            ┆          ┆           ┆ 810              ┆            ┆ Carolina Ave N…  ┆           │
│ 31383      ┆ 11       ┆ null      ┆ 1900964406001033 ┆ -77.331794 ┆ N Shore Dr &     ┆ 38.95937  │
│            ┆          ┆           ┆ 780              ┆            ┆ Bandit Loop      ┆           │
│ 31396      ┆ 11       ┆ null      ┆ 1890204129337198 ┆ -77.233583 ┆ Dorr A